# MAT281 - Tarea 2

**Indicaciones**:

* En los **Ejercicio 1-8** puedes utilizar tanto `matplotlib` como `altair` según te parezca más conveniente o cómodo, en ambos casos cada gráfico debe tener elementos mínimos como:
    - Título
    - Nombre de los ejes, leyendas, etc. en formato _amigable_/_humano_, por ejemplo, si la columna del dataframe en cuestión tiene por nombre `casos_confirmados` se espera que el eje del gráfico tenga por nombre `Casos confirmados`.
    - Colores adecuados al tipo de datos.
    - Un tamaño adecuado para ver con facilidad en una pantalla con resolución HD o FullHD.
    - Cada vez que no se cumplan alguna de estos requerimientos se descontará __1 punto__ de la nota final.

* Para el **Ejercicio 9** es obligación utilizar `altair`.
* Cada ejercicio debe estar acompañado con una celda con comentarios o análisis que puedas desprender de los gráficos.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import altair as alt
import ipywidgets as widgets

from datetime import date
from ipywidgets import interactive, interact

pd.set_option('display.max_columns', 999)
alt.data_transformers.enable('data_server')
alt.themes.enable('opaque')

%matplotlib inline

**COVID-19 en Chile** 

En esta tarea exploraremos los datos de Covid-19 en Chile a profundidad. Las siguientes celdas cargarán los datos a utilizar en tu sesión. Es importante que leas la documentación de cada conjunto de datos para comprender las columnas.

In [2]:
start_date = pd.to_datetime("2020-04-13")

In [3]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto6
confirmados = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto6/bulk/data.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .assign(fecha=lambda x: pd.to_datetime(x["fecha"]))
    .loc[lambda x: x["fecha"] >= start_date]
    .dropna()
    .astype({"casos_confirmados": np.float, "tasa": np.float})
)

confirmados.head()

,poblacion,casos_confirmados,fecha,region_id,region,provincia_id,provincia,comuna_id,comuna,tasa
0,247552.0,270.0,2020-04-27,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,109.067994
1,247552.0,9138.0,2020-10-12,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,3691.345657
2,247552.0,596.0,2020-05-29,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,240.757497
3,247552.0,6131.0,2020-08-17,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,2476.651370
4,247552.0,353.0,2020-05-11,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,142.596303


In [4]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto19
activos = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto19/CasosActivosPorComuna.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .loc[lambda x: x["codigo_comuna"].notnull()]
    .melt(id_vars=["region", "codigo_region", "comuna", "codigo_comuna", "poblacion"], var_name="fecha", value_name="casos_activos")
    .assign(fecha=lambda x: pd.to_datetime(x["fecha"]))
    .loc[lambda x: x["fecha"] >= start_date]
)

activos.head()

,region,codigo_region,comuna,codigo_comuna,poblacion,fecha,casos_activos
0,Arica y Parinacota,15,Arica,15101.0,247552.0,2020-04-13,88.0
1,Arica y Parinacota,15,Camarones,15102.0,1233.0,2020-04-13,0.0
2,Arica y Parinacota,15,General Lagos,15202.0,810.0,2020-04-13,0.0
3,Arica y Parinacota,15,Putre,15201.0,2515.0,2020-04-13,0.0
4,Tarapaca,1,Alto Hospicio,1107.0,129999.0,2020-04-13,8.0


In [5]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto14
fallecidos = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto14/FallecidosCumulativo.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .melt(id_vars=["region"], var_name="fecha", value_name="fallecidos")
    .assign(
        fecha=lambda x: pd.to_datetime(x["fecha"]),
    )
    .loc[lambda x: x["fecha"] >= start_date]
)

fallecidos.head()

,region,fecha,fallecidos
374,Arica y Parinacota,2020-04-13,1.0
375,Tarapacá,2020-04-13,0.0
376,Antofagasta,2020-04-13,1.0
377,Atacama,2020-04-13,0.0
378,Coquimbo,2020-04-13,0.0


In [6]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto10
fallecidos_etareo = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto10/FallecidosEtario.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .melt(id_vars=["grupo_de_edad"], var_name="fecha", value_name="fallecidos")
    .assign(
        fecha=lambda x: pd.to_datetime(x["fecha"]),
        grupo_de_edad=lambda x: x["grupo_de_edad"].str.replace("<=39", "0-39")
    )
    .loc[lambda x: x["fecha"] >= start_date]
)

fallecidos_etareo.head()

,grupo_de_edad,fecha,fallecidos
28,0-39,2020-04-13,3
29,40-49,2020-04-13,2
30,50-59,2020-04-13,3
31,60-69,2020-04-13,12
32,70-79,2020-04-13,29


## Ejercicio 1

(10 puntos)

Mostrar cantidad de fallecidos la fecha por cada grupo etáreo.

In [13]:
fallecidos_etareo['grupo_de_edad']
alt.Chart(fallecidos_etareo).mark_bar().encode(
    
    x = alt.X('grupo_de_edad:O', title =  'Grupo etáreo'),
    y = alt.Y('fallecidos:Q', title = 'Fallecidos'),
    
).properties(title="Fallecidos por Grupo etario",
    width=800, 
    height=400 
)


alt.Chart(...)

**Comentarios:** Se observa que los jovenes son menos propenso a fallecer pot esta pandemia, en cambio el grupo de 70-79 tienen mas fallecidos en esta pandemia, y es congruente con la informacion que se tiene, pues son personas de alto riesgo

## Ejercicio 2

(10 puntos)

¿Qué tan variable es la población de las comunas de Chile? Considera utilizar un gráfico que resuma de buena forma la información sin agregar la variable de región o provincia.

In [261]:
alt.Chart(activos).mark_bar().encode(
    x = alt.X('comuna:O', 
              title = 'Comuna', 
              sort= '-y'),
    y = alt.Y('poblacion:Q', 
              title = 'Población'),
    
).properties(title="Población por comunas", 
    width=1500, 
    height=400 
)


alt.Chart(...)

**Comentarios:** Podemos observar que la comuna de Puente Alto es la comuna con mayor poblacion y rio verde es la comuna con menor habitante. Ademas la mayoria de las comunas tiene una población menor de 150000 personas.

## Ejercicio 3

(10 puntos)

Mostrar evolución y comparación de los fallecimientos entre distintos grupos etáreos, pero que al mismo tiempo sea fácil identificar la cantidad de fallecidos total en cada fecha.

In [42]:
nearest = alt.selection(type='single', nearest=True, on='mouseover',  #Se crea la seleccion de tal manera que elija el punto mas proximo y lo seleccione en funcion del valor x
                        fields=['fecha'], empty='none') 

line = alt.Chart(fallecidos_etareo).mark_line(interpolate='basis').encode(    
    x= alt.X('fecha:T', title = 'Fecha'),
    y= alt.Y('fallecidos', title = 'Fallecidos'),
    color= alt.Color('grupo_de_edad', title = 'Grupo etáreo', scale = alt.Scale(scheme = 'dark2')),
)

selectors = alt.Chart(fallecidos_etareo).mark_point().encode(                  #Valor x del cursor con selectores trasnparente en todo el grafico
    x='fecha',
    opacity=alt.value(0),
).add_selection(
    nearest
)

points = line.mark_point().encode(                                          #se resaltan los puntos en la linea cuando se mueve el cursos, segun la seleccion 
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)

text = line.mark_text(align='left', dx=-30, dy=-5).encode(                  #Etiqueta del texto
    text=alt.condition(nearest, 'fallecidos:Q', alt.value(''))
)


rules = alt.Chart(fallecidos_etareo).mark_rule(color='gray').encode(        #Regla de ubicacion de la seleccion 
    x='fecha:T',
).transform_filter(
    nearest
)

alt.layer(                                                                  # se muestra el grafico con todo lo anterior
    line, selectors, points, rules, text
).properties(title = 'Fallecidos por grupo etáreo VS Fecha',
             width = 800,
             height= 400
)

alt.LayerChart(...)

**Comentarios:** Se observa que hasta el 31 de Mayo los fallecidos por grupo etáero se compotaban casi de forma lineal, y muy similar,  pero despues de esa fecha las curvas no estan tan amortiguadas. Por ultimo a mediados de julio se observa una diferencia 'salto' en las curvas que se puede deber al cambio de notificiación de los fallecidos, debido al cambio de ministro de salud. 

## Ejercicio 4

(10 puntos)

Mostrar en tres gráficos la evolución de casos confirmados, evolución de fallecimientos y evolución de casos activos.

In [55]:
#Creacion de data para casos activos, fallecidos y confirmados

data_activos = activos.drop(['region', 'codigo_region', 'comuna', 'codigo_comuna', 'poblacion'], axis = 1)
data_confirmados = confirmados.drop(['poblacion', 'region_id', 'region', 'provincia_id', 'provincia', 'comuna_id', 'comuna', 'tasa'], axis = 1)
data_fallecidos = fallecidos.drop(['region'], axis = 1)


#Graficos

grafico_activos = alt.Chart(data_activos).mark_line().encode(
    x = alt.X('fecha:T', title = 'Fecha'), 
    y=alt.Y("sum(casos_activos)", title="Casos activos")
).properties(title="Evolución casos activos",
             width = 800,
             height= 400
)

grafico_confirmados= alt.Chart(data_confirmados).mark_line().encode(
    x = alt.X('fecha:T', title = 'Fecha'), 
    y=alt.Y("sum(casos_confirmados)", title="Casos confirmados")
).properties(title="Evolución casos confirmados",
             width = 800,
             height= 400
)

grafico_fallecidos = alt.Chart(data_fallecidos).mark_line().encode(
    x = alt.X('fecha:T', title = 'Fecha'), 
    y=alt.Y("sum(fallecidos)", title="Casos fallecidos")
).properties(title="Evolución casos fallecidos",
             width = 800,
             height= 400
)

grafico_confirmados & grafico_fallecidos   & grafico_activos

alt.VConcatChart(...)

**Comentarios:** Notemos que las graficas de evolución casos fallecidos y evolucion casos confirmados tienen una grafica muy simiiar lo que puede indicar una correlación, aeemas se obsrva el salto a mediados de julio, lo que explicamos anterioremente. 

## Ejercicio 5

(10 puntos)

Comparar la tasa de incidencia entre las regiones a lo largo del tiempo.

In [287]:
data_conf= confirmados.groupby(["region", "fecha"]).apply(lambda x: x["casos_confirmados"].sum()*100000/x["poblacion"].sum()
    ).to_frame('tasa').reset_index()

alt.Chart(data_conf).mark_line().encode(
    alt.X('fecha', axis = alt.Axis(title='Fecha')),
    alt.Y('tasa', title="Tasa de incidencia"),
    
    alt.Color('region', title="Región", scale=alt.Scale(scheme='category20')),
    tooltip = [alt.Tooltip('region', title = 'Region'),
               alt.Tooltip('tasa', title = 'Tasa de incidencia')
    ]
).properties(
    title='Tasa de incidencia por región',
    width = 800,
    height = 400
    
)

alt.Chart(...)

**Comentarios:** Se observa que la region de Magallanes tiene la mayor tasa de incidencia

## Ejercicio 6

(10 puntos)

¿Hay alguna conclusión que puedas obtener rápidamente al graficar un _scatter plot_ con los casos confirmados y tasa de incidencia de cada comuna para los días 13 de abril y 6 de noviembre del 2020? Además, colorea cada punto según la región a la que pertenece y considera si es útil en el gráfico que el tamaño sea proporcional a la población.

In [278]:
confirmados_copy=confirmados.copy()

fecha_inicio = pd.to_datetime("2020-04-13")
fecha_fin= pd.to_datetime("2020-11-06")

mask1=(confirmados_copy['fecha'] == fecha_inicio)  
mask2=(confirmados_copy['fecha'] == fecha_fin)      #filtrando por la fechas dadas


data_nueva = confirmados_copy.loc[mask1 | mask2 ]   #filtrando por la fechas dadas


alt.Chart(data_nueva).mark_circle().encode(
    x = alt.X('casos_confirmados', axis = alt.Axis(title='Casos confirmados')),
    y = alt.Y('tasa', title = 'Tasa de incidencia'),
    
    color = alt.Color('region', title = 'Regiones'),                       #graficando lo pedido, coloreando seun la region que pertenece 
    tooltip = [alt.Tooltip('region', title = 'Region'),
               alt.Tooltip('comuna', title = 'Comuna'),
               alt.Tooltip('casos_confirmados', title = 'Casos confirmados'),
               alt.Tooltip('tasa', title = 'Tasa de incidencia '),
    ],
                                                     
).properties(title = 'Tasa de incidencia  V/S Casos confirmados',
             width = 800,
             height = 400)






alt.Chart(...)

In [276]:
alt.Chart(data_nueva).mark_circle().encode(
    x = alt.X('casos_confirmados', axis = alt.Axis(title='Casos confirmados')),
    y = alt.Y('tasa', title = 'Tasa de incidencia '),
    
    color = alt.Color('region', title = 'Regiones'),                       #graficando lo pedido, coloreando seun la region que pertenece  
    tooltip = [alt.Tooltip('region', title = 'Region'),
               alt.Tooltip('comuna', title = 'Comuna'),
               alt.Tooltip('casos_confirmados', title = 'Casos confirmados'),
               alt.Tooltip('tasa', title = 'Tasa de incidencia'),
    ],
    size = 'poblacion'                                                     # el tamaño sea proporcional a la poblacion
).properties(title = 'Tasa de incidencia  V/S Casos confirmados',
             width = 800,
             height = 400)

alt.Chart(...)

**Comentarios:** si se considera  útil en el gráfico que el tamaño sea proporcional a la población, pues es mas entendible como se observa. Ademas se puede observar que las regiones que tienen mas poblacion, mas casos confirmados tiene. 

## Ejercicio 7

(10 puntos)

1. Grafica la evolución de los casos activos de cada comuna en un solo gráfico. 
2. Grafica la evolución de los casos activos de cada comuna en gráficos separados por región.

Entrega los pros y contras de cada uno de estos enfoques.

In [105]:
#Grafico de la evolucion de los casos activos por comunas

alt.Chart(activos).mark_line().encode(
    x=alt.X('fecha',axis=alt.Axis(title='Fecha')),
    y=alt.Y('casos_activos:Q', axis=alt.Axis(title = 'Casos activos')),
    color =alt.Color('comuna', title = 'Comunas'),
    tooltip=[alt.Tooltip('comuna', title = 'Comuna'),
             alt.Tooltip('casos_activos', title = 'Casos activos')]   
).properties(
    title='Evolución de casos activos por comunas',
    width=800, 
    height=400
)


alt.Chart(...)

In [112]:
#Grafica la evolución de los casos activos de cada comuna en gráficos separados por región.

alt.Chart(activos).mark_line().encode(
    x=alt.X('fecha:T', title =  'Fecha'),
    y=alt.Y("casos_activos",axis = alt.Axis(title = 'Casos activos')),
    color='comuna',
    tooltip = [alt.Tooltip('region', title = 'Region'),
               alt.Tooltip('comuna', title = 'Comuna'),
               alt.Tooltip('casos_activos', title = 'Casos activos')
               ]
).properties(
    title='Evolución de casos activos de comunas Separado por region',
    width=250,
    height=250
).facet(
        title='Evolución de casos activos de Separado por region',
    facet='region:N',
    columns=4
)


alt.FacetChart(...)

**Comentarios:** 

De la primea grafica: \
    -Pros:\
        Se observan a simple vista la comuna con mayor casos activo\
        Se puede obtner una idea de lo que esta pasando en el largo plazo

   -Contras:\
        se solapan los datos\
        no se identifica cada comuna, pues son muchas

De la Segunda grafica: \
    -Pros:\
        Se aprecia una tendencia por regiones \
        Como hay menos comunas por grafico, se puede hacer una analisis mas profundo 

   -Contras:\
        No podemos comparar entre rediones\
        no se identifica la comuna con mayor casos activo, tras una primera mirada 
 



## Ejercicio 8

(10 puntos)

Hacer un gráfico que permita comparar rápidamente entre regiones su promedio de casos activos , máximo de casos confirmados y fallecidos. Utiliza los valores reales y apoyarlos con colores.

Se adjunta el diccionario `region_names` con tal de reemplazar los nombres de las regiones en los datos `fallecidos` para poder unir con los otros datos. 

In [114]:
region_names = {
    "Araucanía": "La Araucanía",
    "Aysén": "Aysén del General Carlos Ibáñez del Campo",
    "Magallanes": "Magallanes y de la Antártica Chilena",
    "Metropolitana": "Metropolitana de Santiago",
    "O’Higgins": "Libertador General Bernardo O'Higgins",
    "Biobio": "Biobío",
    "La Araucania": "La Araucanía",
    "Del Libertador General Bernardo O’Higgins": "Libertador General Bernardo O'Higgins",
    "Los Rios": "Los Ríos",
    "Magallanes y la Antartica": "Magallanes y de la Antártica Chilena",
    "Tarapaca": "Tarapacá",
    "Valparaiso": "Valparaíso",
    "Nuble": "Ñuble",
    "Aysen": "Aysén del General Carlos Ibáñez del Campo",
}

In [222]:
activo_s = (activos.drop(['codigo_region','comuna','codigo_comuna','poblacion','fecha'],axis=1).groupby("region").
            mean().reset_index().replace({"region": region_names}))



confirmado_s = (confirmados.drop(['poblacion','fecha','region_id','provincia_id','provincia','comuna_id','comuna','tasa'], 1).
                groupby("region").max().reset_index().replace({"region": region_names}))

fallecido_s =(fallecidos.drop(['fecha'], axis=1).groupby("region").
              max().reset_index().drop(14).reset_index().replace({"region": region_names}))


dateframe_region = pd.merge(activo_s,confirmado_s, on=['region'])
dateframe_region=pd.merge(dateframe_region,fallecido_s, on=['region']).drop(['index'], axis=1)
dateframe_region

,region,casos_activos,casos_confirmados,fallecidos
0,Antofagasta,78.112795,12831.0,562.0
1,Arica y Parinacota,100.310606,10530.0,207.0
2,Atacama,33.713805,4820.0,106.0
3,Aysén del General Carlos Ibáñez del Campo,4.406061,1086.0,14.0
4,Biobío,46.290174,6401.0,607.0
5,Coquimbo,33.002020,5099.0,270.0
6,Libertador General Bernardo O'Higgins,24.648760,9046.0,544.0
7,La Araucanía,16.421402,5836.0,229.0
8,Los Lagos,24.295455,7812.0,203.0
9,Los Ríos,17.915404,2940.0,55.0


In [255]:

grafico_activos = alt.Chart(dateframe_region).mark_circle(size=100).encode(
    x = alt.X('region:O', title = 'Regiones'),
    y = alt.Y('casos_activos:Q', title = 'casos_activos'),
    color = alt.value('red'),
    tooltip = [alt.Tooltip('region', title = 'Region'),
               alt.Tooltip('casos_activos', title = 'Promedio de casos activos') 
              ]
)

grafico_confirmados = alt.Chart(dateframe_region).mark_circle(size=100).encode(
    x = alt.X('region:O', title = 'Regiones'),
    y = alt.Y('casos_confirmados:Q', title = 'casos_confirmados'),
    color = alt.value('green'),
    tooltip = [alt.Tooltip('region', title = 'Region'),
               alt.Tooltip('casos_confirmados', title = 'Maximo de casos confirmados')   
              ]
    
)

grafico_fallecidos = alt.Chart(dateframe_region).mark_circle( size=100).encode(
    x = alt.X('region:O', title = 'Regiones'),
    y = alt.Y('fallecidos:Q', title = 'fallecidos'),
    color = alt.value('blue'),
    tooltip = [alt.Tooltip('region', title = 'Region'),
               alt.Tooltip('fallecidos', title = 'Maximo de fallecidos')
              ]
    
)


alt.layer(
    grafico_activos, grafico_confirmados, grafico_fallecidos
).properties(title = 'Maximo de fallecidos, Maximo de casos confirmados y Promedio de casos activos por region ',
             width = 1000,
             height= 1000
            )

alt.LayerChart(...)

**Comentarios:** La region con mayor casis confirmados es la region Metropolitana, por mucha diferencia con las otras regiones  y la region de Aysen es la que tiene menor casos confirmados con respecto  las otras regiones. Ademas se observa a simple vista que los maximos de casos confirmados es siempre mayor al promedio de casos activos y maximos fallecidos. 



## Ejercicio 9


En este ejercicio buscaremos realizar un mini-dashboard respecto al estado de los casos de COVID-19 en Chile, por lo tanto utilizaremos haremos uso de datos geográficos de manera operacional (es decir, no nos preocuparemos de proyecciones en mapas ni nada por el estilo), lo único es que debes instalar `geopandas` en tu ambiente virtual y no olvidar actualizarlo en tu `environment.yml` para luego subirlo a tu repositorio de GitHub.

Con tu ambiente activo (`conda activate mat281`) basta con ejecutar `conda install -c conda-forge geopandas` para instalar `geopandas`.

In [256]:
import geopandas as gpd
from pathlib import Path

In [257]:
shp_filepath = Path().resolve().parent / "data" / "regiones_chile.shp"
regiones = gpd.read_file(shp_filepath)
regiones.head()

,Region,objectid,cir_sena,codregion,area_km,st_area_sh,st_length_,geometry
0,Región Metropolitana de Santiago,1092,7,13,15392.030737,2.225204e+10,1.064253e+06,"POLYGON ((-7873736.745 -3885505.642, -7873695...."
1,Región de Antofagasta,1086,3,2,126071.431981,1.508452e+11,2.516112e+06,"MULTIPOLYGON (((-7874671.129 -2977676.850, -78..."
2,Región de Arica y Parinacota,1084,1,15,16866.819844,1.886869e+10,7.505296e+05,"POLYGON ((-7727277.278 -1997230.768, -7726464...."
3,Región de Atacama,1089,4,3,75661.248635,9.643906e+10,2.401741e+06,"MULTIPOLYGON (((-7900342.628 -3153340.296, -79..."
4,Región de Aysén del Gral.Ibañez del Campo,1088,14,11,106703.377369,2.242743e+11,4.144481e+07,"MULTIPOLYGON (((-8208500.834 -5733817.475, -82..."


In [258]:
type(regiones)

geopandas.geodataframe.GeoDataFrame

Lo único que tienes que saber es que un `GeoDataFrame` es idéntico a un `DataFrame` salvo que debe poseer una columna llamada `geometry` caracterice los elementros geométricos, que en este casos son polígonos con los límites de las regiones de Chile. 

Para graficar mapas en Altair se debe usar `mark_geoshape`, además, para no preocuparnos de las proyecciones si o si debes declarar lo siguiente que se muestra en la siguiente celda en las propiedades del gráfico. El resto es igual a cualquier otro gráfico de Altair.

In [259]:
alt.Chart(regiones).mark_geoshape().encode(
).properties(
    projection={'type': 'identity', 'reflectY': True},
    width=250,
    height=600
)

alt.Chart(...)

### Ejercicio 9.1

(10 puntos)

Define el `DataFrame` con el nombre `casos_geo` tal que tenga las columnas

* `region`
* `codigo_region`
* `fecha`
* `poblacion`
* `casos_confirmados`
* `tasa`
* `casos_activos`
* `fallecidos`
* `geometry`

Ten mucho cuidado como unes los dataframes `confirmados`, `activos`, `fallecidos` y `regiones`. Idealmente utilizar el código de región, pero en caso que no se encuentren disponibles utilizar el nombre de la región (no olivdar utilizar el diccionario `region_names`).

In [ ]:
casos_geo = (
    # FIX ME #
)

casos_geo.head()

Ejecuta lo siguiente para convertir el DataFrame anterior en un GeoDataFrames

In [ ]:
casos_geo = casos_geo.pipe(lambda x: gpd.GeoDataFrame(x, geometry="geometry"))

### Ejercicio 9.2

(5 puntos)

Modifica la función `covid_chile_chart` tal que reciba una fecha y una columna. Luego, debe filtrar `casos_geo` con registros de la fecha seleccionada y graficar un mapa donde las regiones se colereen según la columna escogida. 

In [ ]:
def covid_chile_chart(fecha, col):
    
    fecha = pd.to_datetime(fecha)
    data = # FIX ME #
    
    chart = alt.Chart(data).mark_geoshape().encode(
        # FIX ME #
    ).properties(
        projection={'type': 'identity', 'reflectY': True},
        width=150,
        height=400
    )
    
    chart.display()
    return

Prueba con lo siguiente

In [ ]:
fecha = "2020-04-13"
col = "tasa"
covid_chile_chart(fecha, col)

### Ejercicio 9.3

(5 puntos)

Ahora utilizando `widgets` generaremos el dashboard interactivo. Define lo siguiente:

* col_widget: Un `widgets.Dropdown` donde las opciones a seleccionar sean las columnas `poblacion`, `casos_confirmados`, `tasa`, `casos_activos` y `fallecidos`. Además, el argumento `description` debe ser `Columna`.
* fecha_widget: Un `widgets.DatePicker` donde el argumento `description` sea `Fecha`.
* Ambos widgets deben tener el argumento `continuous_update=False`

In [ ]:
import ipywidgets as widgets
from ipywidgets import interactive, interact

In [ ]:
col_widget = # FIX ME #

In [ ]:
fecha_widget = # FIX ME #

Finalmente, haciendo uso de `interactive`, la función `covid_chile_chart` y todos los widgets es posible crear un _dashboard_ interactivo con los datos de Covid-19.

Respira profundo y explora tu creación!

In [ ]:
covid_dashboard = interactive(
    covid_chile_chart,
    fecha=fecha_widget,
    col=col_widget
)
covid_dashboard

**Comentarios:** RESPONDE AQUÍ